## How to play with ECLYPSE

This notebook will walk you through the process of setting up and running your first _simulation_ using ECLYPSE, which is pretty similar to a _simulation_.

In ECLYPSE we distinguish two simulation types:

- **LOCAL**: The simulation runs by placing applications on an infrastructure using specific placement strategies. The applications and infrastructure are updated during the simulation according to their update policies, with updates occurring at each triggered tick event.

- **REMOTE**: The simulation operates like the local one, but the applications are executed on **remote nodes** using [Ray](https://www.ray.io). This allows you to run simulations on a cluster of machines, which can be useful for large-scale simulations. It also performs operations on the services placed on the infrastructure, such as _deploying_, _starting_, _stopping_ and _undeploying_ them.

The main difference between local and remote simulations (for the user, but not under the hood ;) is that in the remote one, **you must provide the logic for each service** in the applications you want to deploy.

To make everything reproducible, we fix a SEED for the simulation. Try to change the SEED and see how the results change!

In [1]:
# Seed for the simulation
SEED = 2

## Build an Infrastructure

The infrastructure represents the computing resources on which the application(s) will be deployed. 
To define an infrastructure, you need to specify its nodes and links, together with the update policy that will be applied to the infrastructure.

First of all, define an update policy that randomly changes the nodes and links resources. 
To do so we just have to define two functions, taking as argument a `networkx.NodeView` and a `networkx.EdgeView`, respectively, and returning the updated set of nodes and edges.

In [2]:
import random as rnd

from networkx.classes.reportviews import (
    EdgeView,
    NodeView,
)


# update edges
def node_random_update(nodes: NodeView):
    for _, resources in nodes.data():
        if rnd.random() < 0.02:
            resources["availability"] = 0
        elif rnd.random() < 0.5 and resources["availability"] == 0:
            resources["availability"] = 1
        else:
            # Randomly update resources with different ranges
            resources["cpu"] = round(max(0, resources["cpu"] * rnd.uniform(0.95, 1.05)))
            resources["gpu"] = round(max(0, resources["gpu"] * rnd.uniform(0.9, 1.1)))
            resources["ram"] = round(max(0, resources["ram"] * rnd.uniform(0.8, 1.2)))
            resources["storage"] = round(
                max(0, resources["storage"] * rnd.uniform(0.9, 1.1))
            )
            resources["availability"] = min(
                1, max(0, resources["availability"] * rnd.uniform(0.995, 1.005))
            )


def edge_random_update(edges: EdgeView):
    for _, _, resources in edges.data():
        # Randomly update resources with different ranges
        resources["latency"] = round(
            max(0, resources["latency"] * rnd.uniform(0.9, 1.1))
        )
        resources["bandwidth"] = round(
            max(0, resources["bandwidth"] * rnd.uniform(0.95, 1.05))
        )

Now you can define the topology, by adding ndoes and links to an extend `networx.Graph`, that is our `eclypse.graph.Infrastructure` class.
The previsouly defined update policies will be parameters of the Infrastructure builder.

In [3]:
from eclypse.graph import Infrastructure

echo_infra = Infrastructure(
    "EchoInfrastructure",
    node_update_policy=node_random_update,
    edge_update_policy=edge_random_update,
    include_default_assets=True,
    seed=SEED,
)
echo_infra.add_node("CloudServer")
echo_infra.add_node("EdgeGateway")
echo_infra.add_node("IoTDevice")
echo_infra.add_node("CloudStorage")
echo_infra.add_node("EdgeSensor")

echo_infra.add_edge(
    "CloudServer", "EdgeGateway", latency=5.0, bandwidth=80.0, symmetric=True
)
echo_infra.add_edge(
    "EdgeGateway", "IoTDevice", latency=8.0, bandwidth=50.0, symmetric=True
)
echo_infra.add_edge(
    "IoTDevice", "CloudStorage", latency=15.0, bandwidth=100.0, symmetric=True
)
echo_infra.add_edge(
    "CloudStorage", "EdgeSensor", latency=9.0, bandwidth=70.0, symmetric=True
)

## Define the Application

The second step is to define the application that will be placed on the infrastructure. Similarly to the infrastructure, the application is defined by a set of components (services) and edges (interactions), and an update policy that will be applied to the application. In this example we do not change the application requirements, so no update policy is defined.

The application is an instance of another extended graph, specifically the `eclypse.graph.Application` class.

**N.B.** When you are in a _remote_ simulation, you must implement each service logic. For simplicity we define one trivial logic that is assigned to all the services.
Such a logic just send an "hello" message to all the neighbours, in both unicast and broadcast mode, printing the received messages and the times to send the messages.
To implement a service logic, you have to extend the `eclypse.remote.service.Service` class and implement the `step` method.

In [4]:
import asyncio
import time

from eclypse.remote.service import Service


class EchoService(Service):
    def __init__(self, id: str):
        super().__init__(id, store_step=True)
        self.i = 0

    async def step(self):
        message = {"message": f"Hello from {self.id}!"}

        neigh = await self.mpi.get_neighbors()
        expected_wait_unicast = 0
        t_init_unicast = time.time()
        for n in neigh:
            req = await self.mpi.send(n, message)
            expected_wait_unicast += req.route.cost(message) if req.route else 0
        t_final_unicast = time.time()
        t_unicast = t_final_unicast - t_init_unicast
        self.logger.info(
            f"Service {self.id}, {self.i} -  Unicasts in: {t_unicast}, expected = {expected_wait_unicast}"
        )
        t_init_broadcast = time.time()
        req = await self.mpi.send(neigh, message)
        expected_wait_broadcast = max(
            [r.cost(message) for r in req.routes if r], default=0
        )
        t_final_broadcast = time.time()
        t_broadcast = t_final_broadcast - t_init_broadcast
        self.logger.info(
            f"Service {self.id}, {self.i} - Broadcasts in: {t_broadcast}, expected = {expected_wait_broadcast}"
        )
        self.i += 1
        await asyncio.sleep(1)
        return (
            self.i,
            t_unicast,
            expected_wait_unicast,
            t_broadcast,
            expected_wait_broadcast,
        )


In the following, we define the `EchoApp` application with 5 services and 4 interactions.

**N.B.** To add a service we use the `add_service` method, by passing the instance of the service that will wrap the logic.

If you want to run a _local_ simulation you can either leave the code as it is, or use the `add_node` method from NetworkX to add a service to the application, specifying just the service name as a string.

In [5]:
from eclypse.graph import Application

echo_app = Application("EchoApp", include_default_assets=True)

echo_app.add_service(
    EchoService("Gateway"),
    cpu=1,
    gpu=0,
    ram=0.5,
    storage=0.5,
    availability=0.9,
    processing_time=0.1,
)

echo_app.add_service(
    EchoService("SecurityService"),
    cpu=2,
    gpu=0,
    ram=4.0,
    storage=2.0,
    availability=0.8,
    processing_time=2.0,
)

echo_app.add_service(
    EchoService("LightingService"),
    cpu=1,
    gpu=0,
    ram=2.0,
    storage=5.0,
    availability=0.8,
    processing_time=1.0,
)

echo_app.add_service(
    EchoService("ClimateControlService"),
    cpu=2,
    gpu=0,
    ram=3.0,
    storage=8.0,
    availability=0.85,
    processing_time=1.5,
)


echo_app.add_service(
    EchoService("EntertainmentService"),
    cpu=3,
    gpu=1,
    ram=4.0,
    storage=10.0,
    availability=0.9,
    processing_time=5.0,
)

echo_app.add_edge(
    "Gateway",
    "LightingService",
    latency=100.0,
    bandwidth=20.0,
    symmetric=True,
)

echo_app.add_edge(
    "Gateway",
    "ClimateControlService",
    latency=100.0,
    bandwidth=10.0,
    symmetric=True,
)

echo_app.add_edge(
    "Gateway",
    "SecurityService",
    latency=50.0,
    bandwidth=5.0,
    symmetric=True,
)

echo_app.add_edge(
    "SecurityService",
    "EntertainmentService",
    latency=50.0,
    bandwidth=10.0,
    symmetric=True,
)

## Configure and run the simulation

After you define your application(s) and infrastructure, ou need to **configure** your simulation by specifying parameters such as its duration, the report and logging features, and other additional settings. 
To do so, it is necessary to define an instance of the `eclypse.simualation.SimulationConfig` class, specifying all possible parameters in the constructor.

For example, if you want to run a **remote** simulation lasting 100 ticks, each every 250ms, save the report in `./my-first-simulation` folder and set a _seed_ for pseudo-randomness, you will have to configure the simulation as follows.
To run it locally (and not remotely), simly set the `remote` parameter to `False`.

**N.B.** We introduced a delay between ticks, to allow services to execute their logics. You can remove it when trying the _local_ simulation.

In [6]:
from eclypse.simulation import SimulationConfig

sim_config = SimulationConfig(
    seed=SEED,
    max_ticks=100,
    tick_every_ms=250,
    path="./my-first-simulation",
    remote=True,
    )

Then create an instance of the `eclypse.simulation.Simulation` class, passing the infrastructur and the configuration as arguments:

In [ ]:
from eclypse.simulation import Simulation

simulation = Simulation(
    infrastructure=echo_infra,
    simulation_config=sim_config,
)

2025-05-09 09:44:08,638	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-09 09:44:09,570	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2025-05-09 09:44:10,141	WARNING actor.py:674 -- Creating an actor from class RemoteSimulator overwrites attribute remote of that class


(RemoteNode pid=1772) 09:44:10 | 🌘 ECLYPSE | EdgeGateway - Node EdgeGateway created.
(RemoteNode pid=1774) 09:44:10 | 🌘 ECLYPSE | CloudStorage - Node CloudStorage created.
(RemoteNode pid=1771) 09:44:10 | 🌘 ECLYPSE | CloudServer - Node CloudServer created.
(RemoteNode pid=1773) 09:44:10 | 🌘 ECLYPSE | IoTDevice - Node IoTDevice created.
(RemoteNode pid=1775) 09:44:10 | 🌘 ECLYPSE | EdgeSensor - Node EdgeSensor created.
(RemoteSimulator pid=1787) 09:44:11 | 🌘 ECLYPSE | Simulation - Event Start-0 fired.
(RemoteSimulator pid=1787) 09:44:11 | 🌘 ECLYPSE | Simulation - Event Tick-0 fired.
(RemoteSimulator pid=1787) 09:44:11 | 🌘 ECLYPSE | Simulation - Event Enact-0 fired.
(RemoteSimulator pid=1787) 09:44:11 | WARNING | EchoInfrastructure - Node "CloudServer" not respected
(RemoteSimulator pid=1787) 09:44:11 | WARNING | EchoInfrastructure - [storage] featured 4 | required 8.0
(RemoteSimulator pid=1787) 09:44:12 | 🌘 ECLYPSE | Simulation - Event Tick-1 fired.
(RemoteSimulator pid=1787) 09:44:12 | 

Now that you have a Simulation object, you can include the application(s) in the simulation by calling the `register` method, passing the application object as an argument, and specifying the **placement strategy** to use. In this case we use one of the off-the-shelf placement strategies provided by ECLYPSE, the `eclypse.placement.strategies.RandomStrategy`:

In [8]:
from eclypse.placement.strategies import RandomStrategy

strategy = RandomStrategy(seed=SEED)

simulation.register(echo_app, placement_strategy=strategy)

Finally we can run the simulation by starting it with the `start` method, then waiting for it to finish with the `wait` method:

In [9]:
simulation.start()
simulation.wait()

Once the simulation is finished, you can access the results by calling the `report` property on the simulation object. This will return a `eclypse.report.Report` object, a friendly interface to select and filter the results of the simulation, that will retrieve pandas dataframes for further analysis.

For instance if you want to get only the metrics related to the application, you can use the `application` method on the report object:

In [10]:
simulation.report.application()

FileNotFoundError: No CSV files found at "my-first-simulation-20250509_094408/csv."